In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [ ]:
class HopfieldNetwork:
    def __init__(self, size):
        self.size = size
        self.weights = np.zeros((size, size))
    
    def train(self, patterns):
        """ Trénování Hopfieldovy sítě na binárních vzorech """
        for p in patterns:
            self.weights += np.outer(p, p)
        self.weights /= len(patterns)  # Normalizace
        np.fill_diagonal(self.weights, 0)  # Diagonála na 0

    def recall_step(self, pattern, mode='synchronous'):
        """ Jeden krok obnovy vzoru """
        recalled_pattern = np.copy(pattern)
        if mode == 'synchronous':
            recalled_pattern = np.sign(self.weights @ recalled_pattern)
        elif mode == 'asynchronous':
            i = np.random.randint(self.size)  # Náhodně vybereme neuron
            recalled_pattern[i] = np.sign(np.dot(self.weights[i], recalled_pattern))
        return recalled_pattern

In [ ]:
# Definice vzorů (5x5 binární matice jako vektor)
patterns = np.array([
    [-1, -1, 1, 1, -1,
     -1, 1, -1, 1, -1,
      1, -1, -1, -1, 1,
     -1, 1, -1, 1, -1,
     -1, -1, 1, 1, -1],
    [1, 1, -1, -1, 1,
     1, -1, 1, -1, 1,
     -1, 1, 1, 1, -1,
     1, -1, 1, -1, 1,
     1, 1, -1, -1, 1],
    [-1, 1, 1, 1, -1,
     1, -1, -1, -1, 1,
     1, -1, -1, -1, 1,
     1, -1, -1, -1, 1,
     -1, 1, 1, 1, -1]
])

In [ ]:
# Inicializace Hopfieldovy sítě
hopfield_net = HopfieldNetwork(size=25)
hopfield_net.train(patterns)

In [ ]:
# Zobrazení všech trénovacích vzorů
fig, axes = plt.subplots(1, len(patterns), figsize=(len(patterns) * 2, 2))
for i, pattern in enumerate(patterns):
    ax = axes[i] if len(patterns) > 1 else axes
    ax.imshow(pattern.reshape(5, 5), cmap='gray', vmin=-1, vmax=1)
    ax.set_title(f'Pattern {i+1}')
    ax.axis('off')
plt.show()

In [ ]:
# Poškozený vzor
noisy_pattern = np.array([
    -1, -1, 1, -1, -1,
    -1, 1, -1, -1, -1,
     1, -1, -1, -1, 1,
    -1, 1, -1, 1, -1,
    -1, -1, 1, 1, -1
])

In [ ]:
# Inicializace animace
fig, ax = plt.subplots()
pattern_display = noisy_pattern.reshape((5, 5))
im = ax.imshow(pattern_display, cmap="gray", vmin=-1, vmax=1)
ax.set_title("Rekonstrukce vzoru")
ax.axis("off")

In [ ]:
def update(frame):
    global noisy_pattern
    noisy_pattern = hopfield_net.recall_step(noisy_pattern, mode='asynchronous')
    im.set_array(noisy_pattern.reshape((5, 5)))
    return im,

In [ ]:
ani = animation.FuncAnimation(fig, update, frames=30, interval=300, blit=True)
plt.show()